# A Short Tour of Julia

In this lecture we will explore:

- Differential equations with ```DifferentialEquations.jl```
- Probabilistic programming with ```Turing.jl```
- Distributed computation

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.add("DifferentialEquations")
Pkg.add("Plots")
Pkg.add("Catalyst")
Pkg.add("Turing")
Pkg.add("Distributions")
Pkg.add("MCMCChains")
Pkg.add("Distributed")
Pkg.add("BenchmarkTools")

## Differential Equations

The ```DifferentialEquations.jl``` package makes easy to define and solve multiple kinds of differential equations, with multiple solvers available depending on the kind of equation.

In [ ]:
using DifferentialEquations
using Plots

Let us define a simple ordinary differential equation 
$$ \frac{du}{dt} = \alpha u $$
where $\alpha$ is a parameter

In [ ]:
α = 1.02
f(u, p, t) = α*u

By default in the definition of the ODE there are:
- The variable $u$.
- A collection of parameters to the ODE (we will show how to use them shortly).
- The time $t$.

We now need to define the initial state $u_0$ and the time interval for which we want to solve our ODE

In [ ]:
u0 = 0.03
tspan = (0.0, 1.0);

We can now define an ODE problem, which is entirely determined by the ODE, the initial conditions, and the timespan:

In [ ]:
prob = ODEProblem(f, u0, tspan)

To obtain a solution we can simply call ```solve```. As additional arguments we can specify:

- the solver to use (see the [documentation](https://diffeq.sciml.ai/v6.16/solvers/ode_solve/#ode_solve)).
- relative and absolute tolerances (keyword arguments ```reltol``` and ```abstol```, respectively).
- we might also only give a _hint_ to select the solver with, for example, ```alg_hints=[:stiff]```.

In [ ]:
sol = solve(prob)

We can now plot the solution (and the analytical solution for comparison):

In [ ]:
plot(sol, lw=3, xaxis="Time (t)", yaxis="u(t)", legend=false)
plot!(0:0.01:1, t->u0*exp(α*t), lw=3, ls=:dash)

Notice that the structure ```sol``` can be used as a function that, for values that were not computed, provides an interpolation:

In [ ]:
[sol(i) for i ∈ 0:0.1:1]

The solution structure also contains information about the pair $(t,u)$ that were computed during the solution process in ```sol.t``` and ```sol.u```:

In [ ]:
println("t = $(sol.t)\nu = $(sol.u)")

### Parameters

We had one argument of $f$ that was not used: ```p```.

```p``` can be an strucure of any type containing the parameters that we want to use in the differential equation.

In [ ]:
g(u, p, t) = p * u

The value of the parameters can be passed as an argument of the ODE problem

In [ ]:
prob = ODEProblem(g, u0, tspan, α)

Everyting continues to work as before

In [ ]:
sol = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8)
plot(sol,lw=3, xaxis="Time (t)", yaxis="u(t)", legend=false)
plot!(sol.t, t->u0*exp(α*t), lw=3, ls=:dash)

### In-place

Until now we had ```In-place: false``` for all our ODE problem. This means that there is a new allocation every time we need to compute a new value of ```f```. We can avoid the allocation by using an ```in-place``` definition where the value is returned by modifing one of the arguments

In [ ]:
function f!(du, u, p, t)
    du[1] = p * u[1]
end

In [ ]:
prob_inplace = ODEProblem(f!, [u0], tspan, α)

There are two things to notice:

- One additional argument (the first argument) to $f!$, which is the value to be modified.
- The value of ```In-place``` when the ```ODEProblem``` is defined is now ```true```

In [ ]:
sol_inplace = solve(prob_inplace)
plot(sol_inplace, lw=3, xaxis="Time (t)", yaxis="u(t)", legend=false)
plot!(0:0.01:1, t->u0*exp(α*t), lw=3, ls=:dash)

### Lotka-Volterra Equations

The Lotka-Volterra equations are a model of population dynamics where there are two species, one acting as prey and one as predator:

$$
\frac{dx}{dt} = \alpha x - \beta xy \\
\frac{dy}{dt} = \delta xy - \gamma y
$$

Here $x$ is the size of the prey population and $y$ the size of the predator population and:

- $\alpha$ is the rate at which the preys increase in number.
- $\beta$ is the rate at which the preys are killed by the predators.
- $\gamma$ is the rate at which the predators die or leave the territory.
- $\delta$ is the rate at which the population of predators increases, that depends on how many preys they are able to catch.

We can now define an (in-place) function for the Lotka-Volterra equations:

In [ ]:
function lotka_volterra!(du, u, p, t)
    α, β, γ, δ = p
    x = u[1]
    y = u[2]
    du[1] = α*x - β*x*y
    du[2] = δ*x*y - γ*y
end

We now model a populations starting at the same size, with a default set of parameters (recall that we can use named tuples to improve readability):

In [ ]:
u0 = [10, 10]
p = (α = 1.1, β = 0.4, γ = 0.4, δ = 0.1)
tspan = (0.0, 100.0)
prob_lv = ODEProblem(lotka_volterra!, u0, tspan, p)

Finding the evolution in time of the two populations can still be obtained by simply calling the ```solve``` function

In [ ]:
sol_lv = solve(prob_lv);

In [ ]:
plot(sol_lv.t, [u[1] for u ∈ sol_lv.u], label="prey")
plot!(sol_lv.t, [u[2] for u ∈ sol_lv.u], label="predator")
xlabel!("time")
ylabel!("population size")

### Others Types of Differential Equations

The library ```DifferentialEquations.jl``` can also manage many kinds of differential equations (from the [documentation](https://diffeq.sciml.ai/v6.16/)):

- Discrete equations (function maps, discrete stochastic (Gillespie/Markov) simulations)
- Ordinary differential equations (ODEs)
- Split and Partitioned ODEs (Symplectic integrators, IMEX Methods)
- Stochastic ordinary differential equations (SODEs or SDEs)
- Random differential equations (RODEs or RDEs)
- Differential algebraic equations (DAEs)
- Delay differential equations (DDEs)
- Stochastic delay differential equations (SDDEs)
- Mixed discrete and continuous equations (Hybrid Equations, Jump Diffusions)
- (Stochastic) partial differential equations ((S)PDEs) (with both finite difference and finite element methods)

We will see as example the SIR model, where the jumps are discrete.

### The SIR model

The SIR model is used to simulate the effect of the diffusion of a disease inside a population. The states of each individual can be:

- Susceptible
- Infected
- Recovered

A susceptible person in contact with an infected one can become infected (with a certain rate $c_1$). An infected person transition to a recovered state with a certain rate $c_2$.

We import the ```Catalyst.jl``` module that allow us to write in a compact way a reaction network. I.e., we want to write that $s + i \rightarrow 2 i$ with a certain rate $c_1$ and $i \rightarrow r$ with a certain rate $c_2$

In [ ]:
using Catalyst

```Catalyst``` makes writing these reactions quite simple:

In [ ]:
sir_model = @reaction_network begin
    c1, s + i --> 2i
    c2, i --> r
    end c1 c2

We can now set the initial conditions:

In [ ]:
p = (1e-4, 0.01)
tspan = (0.0, 300.0)
u0 = [999, 1, 0] # 999 susceptible, 1 infected, 0 recovered
prob_sir = DiscreteProblem(sir_model, u0, tspan, p)

And find the dynamics of the disease:

In [ ]:
prob_jump = JumpProblem(sir_model, prob_sir, Direct())
sol_sir = solve(prob_jump, SSAStepper());

In [ ]:
plot(sol_sir)

### Packages to check

The following packages are related to ```DIfferentialEquations.jl``` and can be interesting for specific applications:

- ```DiffEqFlux.jl```, to create Neural ODE.
- ```diffeqpy``` and ```diffeqr``` makes the solvers of ```DifferentialEquations.jl``` available to Python and R, respectively.

## Probabilistic Programming

One of the main libraries in Julia for probabilistic programming is ```Turing.jl```.

Programming works with a known model with known parameters to generate some data.

Probabilistic programming is the case when the data and the model are known but we ignore the parameters.

#### Some references

[An Introduction to Probabilistic Programming](https://arxiv.org/abs/1809.10756)

Let us import ```Turing.jl``` plus a library of distributions, and a library to work with Makow chain Monte Carlo

In [ ]:
using Turing
using Distributions
using MCMCChains

A model (prefixed with the macro ```@model```) is a function. The function can then be used to condition the model on the data.

Here we model a coin flip (example taken from this [tutorial](https://turing.ml/dev/tutorials/0-introduction/)) where we do not know the probability $p$ of landing on head or tail.

Here, we have $y_1, \ldots, y_n$ samples that we know that theu will be distributed according to a Bernoulli distribution with parameter $p$.

In [ ]:
@model function coin(y)
    p ~ Beta(1,1)
   
    for i ∈ 1:length(y)
        y[i] ~ Bernoulli(p)
    end
end;

We can now generate $100$ samples from a fair coin:

In [ ]:
data = rand(Bernoulli(0.5), 100);

We can then use the model to estimate the probability $p$ of landing on tail given that data that we have:

In [ ]:
ϵ = 0.05
τ = 10
iterations = 1000
chain = sample(coin(data), HMC(ϵ, τ), iterations)

In [ ]:
histogram(chain[:p], label=:none)

## Distributed Computation

We are going to explore how it is possible to easily distribute work across multiple processes (and, possibly, machines) in Julia.

To see size of computations possible in Julia see the presentation [Celeste.jl: Petascale Computing in Julia](https://www.youtube.com/watch?v=uecdcADM3hY).

First of all, we import the ```Distributed``` package

In [ ]:
using Distributed
using BenchmarkTools

We can use the function ```nprocs``` to see the number of active Julia processes

In [ ]:
nprocs()

Additional worker processes can be added with the ```addproc``` function, which return an array of integer ids representing the newly created processes

In [ ]:
addprocs(1)

As we can see, the number of Julia processes is now increased:

In [ ]:
nprocs()

We will see a collection of macros, structures, and functions that are used for computation among multiple processes:

- ```@everywhere``` to execute a block of code in all processes.
- ```@spawnat``` to execute a function in a specific process.
- ```Future``` and ```fetch``` the result of 
- ```@sync```
- ```@distributed```

Our test case will be the following function to approximate $\pi$ via a uniform sampling in $[0,1]^2$.

In [ ]:
function approx_pi(n)
    s = 0
    for i ∈ 1:n
        p = (rand(), rand())
        s += p[1]^2 + p[2]^2 <= 1
    end
    4s/n
end

Let us benchmark it using ```BenchmarkTools.jl```

In [ ]:
@btime approx_pi(10^7)

We can now distribute the computation:

- Define a function ```approx_pi_sum``` on every process
- Split the $n$ points to sample in equal-length chunks and distribute it across processes with ```@spawnat```
- Each computation returns a ```Future```
- Wait for each computation to finish with ```@sync```
- Fetch the results of the computation with ```fetch```
- Finally compute this approximation of $\pi$

In [ ]:
@everywhere function approx_pi_sum(n)
    s = 0
    for i ∈ 1:n
        p = (rand(), rand())
        s += p[1]^2 + p[2]^2 <= 1
    end
    s
end

function compute_pi(n)
    np = nprocs()
    partial_sums = Vector(undef, np)
    k = n ÷ np
    @sync for i ∈ 1:np
        partial_sums[i] = @spawnat i approx_pi_sum(k)
    end
    missings = n - np*k
    s = approx_pi_sum(missings)
    4*(sum(fetch.(partial_sums)) + s)/n
end
     

In [ ]:
@btime compute_pi(10^7)

Since this is so common, we can use the ```@distributed``` macros, that distribute the computation of a ```for``` cycle across all processes and reduce it with a given function

In [ ]:
function distributed_pi(n)
    sum = @distributed (+) for i ∈ 1:n
            p = (rand(), rand())
            p[1]^2 + p[2]^2 <= 1
    end
    4*sum/n
end

In [ ]:
distributed_pi(10^7)

In [ ]:
@btime distributed_pi(10^7)

### Q&A